Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

Create a small validation set.

Utility functions to map characters to vocabulary IDs and back.

Function to generate a training batch for the LSTM model.

Simple LSTM Model.

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [5]:
vocabulary_size = 729

def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
#  print('predictions is %s' % predictions)
#  print('labels is %s' % labels)
#  print('label shape is %s' % str(labels.shape))
#  print('labels[0] shape is %s' % labels[0])
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [6]:
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0

def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '


def chars2eid(chars):
  return char2id(chars[0]) * 27 + char2id(chars[1])

def eid2chars(eid):
  if eid > 0:
    return id2char(eid // 27) + id2char(eid % 27)
  else:
    return '  '

In [7]:
batch_size = 64
num_unrollings = 10
#vocabulary_size = 128   # since we are now projecting our 729 to 128

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size / batch_size
        self._cursor = [ offset * segment for offset in xrange(batch_size)]
        self._last_batch = self._next_batch()
        self._last_label = self._next_batch_label()
  
    
#    def _next_batch(self):
#        batch = np.zeros(shape=(self._batch_size), dtype=np.int32)
#        for b in range(self._batch_size):
#          batch[b] = char2id(self._text[self._cursor[b]])
#          self._cursor[b] = (self._cursor[b] + 1) % self._text_size
#        return batch
    
        
    def _next_batch(self):
  
        batch = np.zeros(shape=(self._batch_size), dtype = np.int32)
        for b in range(self._batch_size):
          c = self._cursor[b]
#          batch[b] = chars2eid(self._text[c:c+2])
          bigram = chars2eid(self._text[c:c+2])
          batch[b] = bigram
          self._cursor[b] = (self._cursor[b] + 2) % self._text_size
        return batch
  
#    def _next_batch_label(self):
#        l = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
#        for b in range(self._batch_size):
#          print('^^^^^^^^')
#          print(self._text[self._cursor[b]])
#          print('==========')
#          print(self._text[self._cursor[b] : (self._cursor[b] + 2)])
#          l[b, chars2eid(self._text[self._cursor[b] : self._cursor[b] + 2])] = 1.0
#        return l
    
    def _next_batch_label(self):
  
        label = np.zeros(shape=(self._batch_size,vocabulary_size), dtype = np.int64)
        for b in range(self._batch_size):
          c = self._cursor[b]
#          batch[b] = chars2eid(self._text[c:c+2])
          bigram = chars2eid(self._text[c:c+2])
          label[b, bigram] = 1.0
          self._cursor[b] = (self._cursor[b] + 2) % self._text_size
        return label
  
    def next(self):
#        batches = []
#        labels = []
        batches = [self._last_batch]
        labels = [self._last_label]
        for step in range(self._num_unrollings):
          batches.append(self._next_batch())
          labels.append(self._next_batch_label())
        
        self._last_batch = batches[-1]
        self._last_label = labels[-1]
        return (batches, labels)
    
def batches2string(batches):
    s = [''] * batches[0].shape[0]

    for b in batches:
   
        s = [''.join(x) for x in zip(s, characters(b))]

    return s

def characters(probabilities):
 
    return [eid2chars(c) for c in np.argmax(probabilities, 1)]

 
train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

test,test_label = train_batches.next()

print(len(test))
#print(test_label)
print('#' * 100)
#print(test[1])
#print(test_label[0][0])
#print(test_label[0][1])

test2,test_label2 = train_batches.next()

print(len(test2))
print('#' * 100)
print(test[10])
print(test2[0])
print(test2[1])

print('#' * 100)
print(test2[0][0])
print(test_label2[0][0])
print(test2[0][1])
print(test_label2[0][1])
print(characters(test_label2[0]))
print([eid2chars(id) for id in test2[0]])

11
####################################################################################################
11
####################################################################################################
[  2 599  22 549 545 139 109  14 154   5 411  23 221 384 495   4 518 324
 548 356 382 378  46 587 495   3   1 548 149   9   1 162  13 135   1 521
  90   3  47 258  15  41   6 356  15 522  20 153  43 513 418 250   2 419
 405 257  46 518 149 533  15 109  18 694]
[  2 599  22 549 545 139 109  14 154   5 411  23 221 384 495   4 518 324
 548 356 382 378  46 587 495   3   1 548 149   9   1 162  13 135   1 521
  90   3  47 258  15  41   6 356  15 522  20 153  43 513 418 250   2 419
 405 257  46 518 149 533  15 109  18 694]
[139 221 561 549 411 154 397 135 149 236  46   9   4 149 423 324 217 487
 135 555  36 167 139 221 540 393 258 423  20 345  20 516 225 383   4  11
  18 432 397 411 424 333 324 217 263 398 450 137   9 444 411 349 513 555
 385 221  81 139 108  46 548 366 491 360]
#########

In [8]:
num_nodes = 64
vocabulary_size = 729
embedding_size = 128
totalGate = 4

forgetGateIndex = 0
inputGateIndex = 1
updateGateIndex = 2
outputGateIndex = 3

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.

  embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))            
  gateWeightInput = tf.Variable(tf.truncated_normal([totalGate, embedding_size , num_nodes] , -0.1 , 0.1));
  gateWeightOutput= tf.Variable(tf.truncated_normal([totalGate, num_nodes , num_nodes] , -0.1 , 0.1));
  gateBias = tf.Variable(tf.zeros([totalGate , 1, num_nodes]));
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
#  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))


  def lstm_cell(i, o, state):
        
    stackedInput = tf.pack([tf.nn.embedding_lookup(gateWeightInput[g,:,:],i) for g in range(totalGate)])
    
    stackedOutput = tf.pack([o for _ in range(totalGate)])
    
    stackedAll = stackedInput+tf.batch_matmul(stackedOutput,gateWeightOutput)+gateBias
        
    listOfGates = tf.unpack(stackedAll)

    forgetGate = tf.sigmoid(listOfGates[forgetGateIndex])
    inputGate = tf.sigmoid(listOfGates[inputGateIndex])
    updateGate = tf.tanh(listOfGates[updateGateIndex])
    outputGate = tf.sigmoid(listOfGates[outputGateIndex])
    
    state = forgetGate * state + inputGate * updateGate    
    
    return outputGate * tf.tanh(state), state

  # Input data.
  train_inputs = list()
  train_labels = list()
  for _ in range(num_unrollings + 1):
    train_inputs.append(tf.placeholder(tf.int32, shape=[batch_size]))
    train_labels.append(tf.placeholder(tf.int32, shape=[batch_size, vocabulary_size]))
  
  train_inputs = train_inputs[:num_unrollings]
  train_labels = train_labels[1:]  # labels are inputs shifted by one time step.
                                                        
                                                        
    
#  train_data = list()
#  for _ in range(num_unrollings + 1):
#    train_data.append(
#      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
#  train_inputs = train_data[:num_unrollings]
#  train_labels = train_data[1:]  # labels are inputs shifted by one time step.


  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits, tf.concat(0, train_labels)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
#  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  sample_input = tf.placeholder(tf.int32, shape=[1])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [14]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):  
    (batches, labels) = train_batches.next()
    
    feed_dict = dict()
   
    for i in range(num_unrollings):
      feed_dict[train_inputs[i]] = batches[i]
      feed_dict[train_labels[i]] = labels[i+1]     
#      feed_dict[train_labels[i]] = labels[i]   
    batches = train_batches.next()
 #   print(batches)
 #   feed_dict = dict()
 #   for i in range(num_unrollings + 1):
 #     feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
#        print(predictions.shape)
#        print(predictions)
#        print(labels.shape)
#        print(labels)
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
#      labels = np.concatenate(list(batches)[1:])
      labels = np.concatenate(list(labels)[1:])
      print(labels)
      print(labels.shape)
      print('prediction: %s' % str(predictions.shape))  
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
      
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          print(sentence)
#          sentence = characters(feed)[0]
          feed2 = np.array([chars2eid(sentence)])
          print(feed2)
          print(feed2.shape)
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed2})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
#      reset_sample_state.run()
#      valid_logprob = 0
#      for _ in range(valid_size):
#        b = valid_batches.next()
#        predictions = sample_prediction.eval({sample_input: b[0]})
#        valid_logprob = valid_logprob + logprob(predictions, b[1])
#      print('Validation set perplexity: %.2f' % float(np.exp(
#        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.591376 learning rate: 10.000000
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 1 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
(640, 729)
prediction: (640, 729)
Minibatch perplexity: 728.78
e 
[135]
(1,)
e pgbvxbwukefrz slrfidlld fviuuwcrkhkzdbzorcgnhdiizqdlramutsrcnblcjccdcgzhzhjsyljmsffbrpcyoxqmddzn nanqidwsmdqgerkguleg dqm espjbydhoapcrwsvkesudwpm wnaqlguercb
qn
[473]
(1,)
qnstgnxopmjtq xvzw  izwzrnibesnjiwqkiejyfehyckvzfjnhmy kqdanrcrkpqvsfdzpnzhxsfwhzzv dereviabqtoblmsexoxjtkakdy gqasookpbzitdelrogrgurlrkacwkphwipxkyud bnncnssxj
pg
[439]
(1,)
pgpiedarfjxxethkrlakpdizetraxplhmw yyvaobtiwlowozpiseuyhgxyhkskgyuonghtsvvovmqngxuniqvvrovojomlolqlmueymvblmlasindtyayywcidcttuplwsgsey reobfbv xhftsrapljp bfra
uw
[590]
(1,)
uwrgs ts tyagbbwauawhlvupxuilzucmbgnsyyopvmbaolpckdqryozxztoekhmgvifmsjtqfsuahsxkwfvapmuvhujdamrpggcydonsehvylmnxjufwve huusauunaywuxyarwz o eiecjrfselkjruf ifu
zb
[704]
(1,)
zbuyppzkcckyldmmwdgizfdyf

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---